In [39]:
from __future__ import division
from sklearn import linear_model
import csv
import numpy as np
import time
import math
import sklearn.metrics
from sklearn import metrics
from sklearn import preprocessing
from sklearn import cross_validation

"""
Applies logistic regression for predicting best picture nomination &
best picture winner
"""

def csvToArray(filename):
    ret = []
    with open(filename) as x:
        entryreader = csv.reader(x, delimiter=',')
        for row in entryreader:
            ret.append(row)
    return ret

def getTrainTestRowsAndCols(labelOfInterest, titleYearIdx):
    """

    :param labelOfInterest: #0 means nominee , 1 means winner (for best picture), 2 means number of awards
    :return:
    """
    trainRows = []
    testRows = []

    for i in range(len(features)):
        if labelOfInterest != 1:
            if int(math.floor(features[i][titleYearIdx])) % 4 == 0:
                testRows.append(i)
            else:
                trainRows.append(i)
        else:
            if labels[i][0] == 1:
                if int(math.floor(features[i][titleYearIdx])) % 4 == 0:
                    testRows.append(i)
                else:
                    trainRows.append(i)

    # prints percentage of train and test
    print len(trainRows) / len(features), len(testRows) / len(features)

    # feature selection based on correlation values
    correlation = csvToArray("../../feature_correlation_results.csv")

    favoriteCols = []
    favoriteColsNames = []

    threshold = 0.1
    for i in range(1, len(correlation)):
        if labelOfInterest == 2 and ('Won' in correlation[i][0]):  # we should ignore this when predicting num of awards
            threshold = 0.1
            continue

        if labelOfInterest == 1 and ('Won' in correlation[i][0]):
            threshold = 0.1
            continue

        if labelOfInterest == 0 and (('Won' in correlation[i][0])):
            threshold = 0.1
            continue

        if correlation[i][0] in featIdxMap:
            if math.fabs(float(correlation[i][1 + labelOfInterest])) > threshold:
                # print correlation[i][0]
                favoriteCols.append(featIdxMap[correlation[i][0]])
                favoriteColsNames.append(correlation[i][0])

    print 'favoritCols = ', len(favoriteCols)

    print favoriteColsNames

    # making test set half positive and half negative
    # removes some of negative instances
    # positive could mean that the instance has been nominated for OR has won best picture!

    return trainRows, testRows, favoriteCols


features = csvToArray("../../features.csv")
featNames = features[0]
features = features[1:]
print len(features), len(features[0])

labels = csvToArray("../../labels.csv")
labelNames = labels[0]
labels = labels[1:]
print len(labels), len(labels[0])

labels = np.array(labels).astype(int)
features = np.array(features).astype(float)

featIdxMap = dict()
for i in range(len(featNames)):
    featIdxMap[featNames[i]] = i

#getting title_year column number
titleYearIdx = -1
for i in range(len(featNames)):
    if featNames[i] == 'title_year':
        titleYearIdx = i
print 'year index = ', titleYearIdx


labelOfInterest = 0
trainRows, testRows, favCols = getTrainTestRowsAndCols(labelOfInterest, titleYearIdx)

for i in range(len(features[0])):
    if (i in favCols):
        features[:, i] = preprocessing.scale(features[:, i])





5043 8175
5043 3
year index =  19
0.758080507634 0.241919492366
favoritCols =  25
['color', 'title_year', 'imdb_score', 'Nominated Actor Leading', 'Nominated Actor Supporting', 'Nominated Actress Leading', 'Nominated Actress Supporting', 'Nominated Art Direction', 'Nominated Cinematography', 'Nominated Costume Design', 'Nominated Directing', 'Nominated Film Editing', 'Nominated Makeup', 'Nominated Music Scoring', 'Nominated Music Song', 'Nominated Sound', 'Nominated Sound Editing', 'Nominated Visual Effects', 'Nominated Writing', 'genre=War', 'genre=Biography', 'genre=History', 'genre=Drama', 'keyword=homosexuality', 'keyword=1920s']


In [40]:
logclf = linear_model.LogisticRegression(C=1e5)
        
logclf.fit((features[trainRows, :])[:,favCols], labels[trainRows,labelOfInterest])

#cross validation
predicted = cross_validation.cross_val_predict(logclf, (features[trainRows, :])[:, favCols], labels[trainRows, labelOfInterest], cv=10)
print 'logistic cross_val_predict(f1-measure) = ', metrics.f1_score(labels[trainRows, labelOfInterest], predicted)
print 'logistic cross_val_predict(f1-measure) = ', metrics.precision_score(labels[trainRows, labelOfInterest], predicted)

# prediction on test
z = logclf.predict((features[testRows, :])[:,favCols])
print 'logistic finalpredict f1-measure = ', metrics.f1_score(labels[testRows, labelOfInterest], z)
print 'logistic finalpredict f1-measure = ', metrics.precision_score(labels[testRows, labelOfInterest], z)




logistic cross_val_predict(f1-measure) =  0.806451612903
logistic cross_val_predict(f1-measure) =  0.842696629213
logistic finalpredict f1-measure =  0.905982905983
logistic finalpredict f1-measure =  0.929824561404


In [28]:
print len(features[0])
a = np.array([])
a = np.append(a, [[1,2]])
a = np.append(a, [[1,2]])
a = np.append(a, [[1,2]])
print a


#nomination prediction:
logistic cross_val_predict(f1-measure) =  0.806451612903
logistic finalpredict f1-measure =  0.905982905983

# win prediction
logistic cross_val_predict(f1-measure) =  0.699029126214
logistic finalpredict f1-measure =  0.685714285714


8175
[ 1.  2.  1.  2.  1.  2.]
